<a href="https://colab.research.google.com/github/angegiulio/IFC-Cost-Calculation/blob/main/NB_Kostensch%C3%A4tzung_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ifcopenshell pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.3 MB/s eta 0:00:00


In [ ]:

ifc_file = ifcopenshell.open(r"NB_AR_SB_M ARCHI-RE.ifc")  # Replace with actual file path


✅ Exported wall data to wall_surface_summary.csv


In [16]:
import ifcopenshell
import pandas as pd
import re

# === Load IFC file ===
ifc_file = ifcopenshell.open(r"NB_AR_SB_M ARCHI-RE.ifc")

# === Helper functions ===

def get_clean_name(name):
    """Strip after the second colon (:) in the name."""
    if not name:
        return None
    parts = name.split(":")
    return ":".join(parts[:2]) if len(parts) > 1 else name


def get_building_of_element(element):
    """Get the building to which the element belongs."""
    for rel in element.ContainedInStructure or []:
        for obj in rel.RelatingStructure.RepresentationContexts or []:
            if obj.is_a("IfcBuilding"):
                return obj.Name
        if rel.RelatingStructure.is_a("IfcBuilding"):
            return rel.RelatingStructure.Name
        elif rel.RelatingStructure.is_a("IfcBuildingStorey"):
            # Go up one level to building
            parent = rel.RelatingStructure.Decomposes
            while parent:
                parent_obj = parent[0].RelatingObject if parent else None
                if parent_obj and parent_obj.is_a("IfcBuilding"):
                    return parent_obj.Name
                parent = parent_obj.Decomposes if parent_obj else None
    return "Unknown"

def get_building_of_element(element):
    """Traverse up containment to find the IfcBuilding."""
    if not element.ContainedInStructure:
        return "Unknown"

    for rel in element.ContainedInStructure:
        structure = rel.RelatingStructure
        while structure:
            if structure.is_a("IfcBuilding"):
                return structure.Name
            if hasattr(structure, "Decomposes") and structure.Decomposes:
                structure = structure.Decomposes[0].RelatingObject
            else:
                break
    return "Unknown"


def get_surface_area(wall):
    """Get wall surface area in square meters."""
    shape = ifcopenshell.geom.create_shape(settings, wall)
    area = shape.geometry.surface.area()
    return area / 1_000_000  # Convert mm2 to m2

# === Geometry settings ===
import ifcopenshell.geom
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# === Extract walls ===
walls = ifc_file.by_type("IfcWall")

data = []

for wall in walls:
    clean_name = get_clean_name(wall.Name)
    building_name = get_building_of_element(wall)
    fire_resistance = get_property_value(wall, "Fire Protection", "Feuerwiderstand Soll")
    try:
        surface = get_surface_area(wall)
    except Exception as e:
        surface = 0.0

    data.append({
        "Building": building_name,
        "WallName": clean_name,
        "FireResistance": fire_resistance,
        "Surface_m2": surface
    })

# === Convert to DataFrame ===
df = pd.DataFrame(data)

# === Group and summarize ===
grouped = df.groupby(["WallName", "FireResistance"]).agg({
    "Surface_m2": "sum"
}).reset_index()

print("\n=== Unique combinations of Wall Name and Fire Resistance with Surface Areas (m²) ===")
print(grouped)

# === Optional: show grouped by building ===
print("\n=== Wall Names grouped by Building ===")
wall_names_by_building = df.groupby("Building")["WallName"].unique()
print(wall_names_by_building)



=== Unique combinations of Wall Name and Fire Resistance with Surface Areas (m²) ===
                      WallName FireResistance  Surface_m2
0       Basic Wall:FAS_VEP_020     EI 60-RF 1         0.0
1       Basic Wall:WAN_GLS_050          EI 30         0.0
2       Basic Wall:WAN_GLS_060           RF 1         0.0
3   Basic Wall:WAN_MWK_BKS_100          EI 30         0.0
4   Basic Wall:WAN_MWK_BKS_100     EI 60-RF 1         0.0
..                         ...            ...         ...
89  Basic Wall:WAT_STB_ORT_380     EI 60-RF 1         0.0
90  Basic Wall:WAT_STB_ORT_400          EI 30         0.0
91  Basic Wall:WAT_STB_ORT_500           RF 1         0.0
92  Basic Wall:WAT_STB_ORT_550           RF 1         0.0
93  Basic Wall:WAT_STB_ORT_810          EI 60         0.0

[94 rows x 3 columns]

=== Wall Names grouped by Building ===
Building
Sanierungsbau    [Basic Wall:WAT_STB_ORT_140, Basic Wall:WAT_ST...
Name: WallName, dtype: object


In [17]:
grouped.to_csv("wall_fire_resistance_summary.csv", index=False)